<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/preprocessing/SolarRadiationCSV_to_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hier werden die Solarstrahlungsdaten aus der Vorhersage von CAMS in einen Dataframe gespeichert:

**Dateiname: SolarRadiationCSV_to_DF**

Die Daten können als CSV-Datei über https://www.soda-pro.com/ heruntergeladen werden.

Log:

10.12.24 - Anpassung des Dataframes um eine aktualisierte Vorhersage.

## Import der benötigten Module und allgemeines Setup:

In [1]:
# Import der erfordelichen Module
import os
import numpy as np
import pandas as pd

In [2]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Pfad zur CSV-Datei auf Google Drive
import_folder = '/content/drive/My Drive/Colab_Notebooks/CSV_SolarRadiation/'
export_folder = '/content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/'

import_file_name = 'CAMS_solar_radiation.csv'
export_file_name = 'CAMS_Glo_CSV'

## Import der Vorhersagewerte:

In [4]:
df_import = pd.read_csv(import_folder + import_file_name , sep = ';')
# Anpassen des Zeitstempels
df_import['Observation_period'] = pd.to_datetime(df_import['Observation_period'].str.slice(0, 19).str.replace('T', ' '))

## Aggregieren der Vorhersagewerte:

Die Daten liegen mit einer minütlichen Auflösung vor und werden im Folgenden zu 2 minütlichen Werten aggregiert.

In [5]:
# Verbinden der Dataframes mit neuen und alten Vorhersagedaten:

df_import.set_index('Observation_period', inplace=True, drop= False)
df_2min = df_import.resample('2min').mean()
df_2min['Observation_period'] = df_2min['Observation_period'] - pd.to_timedelta(.5, unit='min')

In [6]:
columns = ['Clear_sky_GHI', 'Clear_sky_BHI', 'GHI', 'BHI']

df_2min[columns] = df_2min[columns] * 6

In [7]:
df_2min

,Observation_period,Clear_sky_GHI,Clear_sky_BHI,GHI,BHI
Observation_period,,,,,
2022-06-01 00:00:00,2022-06-01 00:00:00,0.0,0.0,0.0,0.0
2022-06-01 00:02:00,2022-06-01 00:02:00,0.0,0.0,0.0,0.0
2022-06-01 00:04:00,2022-06-01 00:04:00,0.0,0.0,0.0,0.0
2022-06-01 00:06:00,2022-06-01 00:06:00,0.0,0.0,0.0,0.0
2022-06-01 00:08:00,2022-06-01 00:08:00,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2023-08-31 23:50:00,2023-08-31 23:50:00,0.0,0.0,0.0,0.0
2023-08-31 23:52:00,2023-08-31 23:52:00,0.0,0.0,0.0,0.0
2023-08-31 23:54:00,2023-08-31 23:54:00,0.0,0.0,0.0,0.0


## Abspeichern der Globalstrahlungs-Vorhersagewerte in einen Dataframe:

In [8]:
def save_dataframe_to_drive(dataframe, folder_path, filename):

    # Erstelle den vollen Pfad zur Datei
    full_path = os.path.join(folder_path, filename)

    # Speichere den DataFrame als CSV-Datei auf Google Drive
    dataframe.to_csv(full_path, index=False)

    print(f'Der DataFrame wurde als {filename} in {folder_path} auf Google Drive gespeichert.')

In [9]:
save_dataframe_to_drive(df_2min, export_folder, export_file_name)

Der DataFrame wurde als CAMS_Glo_CSV in /content/drive/My Drive/Colab_Notebooks/CAMS_Vorhersage/ auf Google Drive gespeichert.
